## Libraries

In [1]:
import pandas as pd
import numpy as np
from crepes import ConformalClassifier, ConformalPredictiveSystem
from crepes.extras import hinge, margin, binning, DifficultyEstimator

## Data

## Calibration 

In [5]:
data_dir = "calibration_and_test.xlsx" #directory
df_cal=pd.read_excel(data_dir,sheet_name='conf_score_25') #cal dataset with conf=0.25
df_cal_tp = df_cal[df_cal['TP']==1] # True Positive Predictions Only

# 43 samples from each class
sample_size = 43
df_sampled = df_cal_tp.groupby('pred_obj_id').apply(lambda x: x.sample(sample_size))
df_sampled = df_sampled.reset_index(drop=True)

df_test=pd.read_excel(data_dir,sheet_name='test_conf_score_25') #test dataset with conf=0.25
df_test_filtered = df_test[df_test['pred_obj_id'] != '-']
df_calibration=df_sampled
df_testing=df_test_filtered

In [6]:
from my_functions import (softmax, 
                          softmax_df, 
                          calculate_logit, 
                          calculate_sigmoid, 
                          calculate_softmax,
                          calculate_hinge_scores,
                          calculate_p_values,
                          calculate_prediction_sets)

In [9]:
df_calibration

,filename,image_width,image_height,gt_obj_id,gt_x_center,gt_y_center,gt_obj_width,gt_obj_height,pred_obj_id,TP,...,pred_y_center,pred_obj_width,pred_obj_height,conf_score,objectness_score,ascaris_conf_score,trichuris_conf_score,hookworm_conf_score,schistosoma_conf_score,ascaris_sig
0,9279375_1367_31_4d4ed9f3-16db-4a1a-b02f-406486...,1280,720,0,0.4145,0.2812,0.0523,0.1014,0,1,...,0.284722,0.048438,0.083333,0.838163,0.84051,0.838163,0.001048,0.005303,0.002017,0.997208
1,8958558_1327_31_4d1b420a-e63f-4de6-b1cc-ad52ac...,1280,720,0,0.7352,0.2785,0.0516,0.1097,0,1,...,0.277083,0.054688,0.109722,0.883463,0.886547,0.883463,0.002676,0.005442,0.001797,0.996521
2,13320612_1851_38_39429b2c-a756-467a-8320-09ac7...,1280,720,0,0.5609,0.9056,0.0625,0.0861,0,1,...,0.903472,0.058594,0.079167,0.919294,0.92068,0.919294,0.002379,0.004979,0.001733,0.998495
3,9264109_1367_31_4d4ed9f3-16db-4a1a-b02f-406486...,1280,720,0,0.6113,0.6472,0.0508,0.0944,0,1,...,0.639583,0.042969,0.095833,0.557859,0.568859,0.557859,0.001498,0.012348,0.008093,0.980663
4,9332647_1380_31_46072060-6150-49f0-9704-a119c7...,1280,720,0,0.8875,0.3708,0.0563,0.0972,0,1,...,0.371528,0.046094,0.104167,0.796945,0.799219,0.796945,0.002139,0.004123,0.001189,0.997155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,12825860_1787_37_27ec8f45-01bf-4087-8b02-26ccd...,1280,720,3,0.6324,0.2597,0.0867,0.2417,3,1,...,0.258333,0.0875,0.255556,0.9637,0.963706,0.000034,0.000516,0.001531,0.9637,0.000035
168,7141784_1125_31_db6784e7-fa75-49aa-804f-464fef...,1280,720,3,0.9059,0.2792,0.1258,0.175,3,1,...,0.275694,0.114844,0.143056,0.888821,0.88888,0.000253,0.000428,0.001991,0.888821,0.000284
169,12822383_1787_37_27ec8f45-01bf-4087-8b02-26ccd...,1280,720,3,0.3191,0.8715,0.1305,0.1625,3,1,...,0.868056,0.128906,0.186111,0.273736,0.274738,0.001445,0.000759,0.001567,0.273736,0.005261
170,8885772_1312_31_8c0444cb-5fd0-46e0-81a5-ae1ece...,1280,720,3,0.8242,0.6326,0.1094,0.2181,3,1,...,0.629167,0.10625,0.244444,0.418137,0.418232,0.000205,0.000498,0.001534,0.418137,0.000491


## Calibration stage

In [10]:
alphas_cal = hinge(
    df_calibration[[
        'ascaris_conf_score',
        'trichuris_conf_score',
        'hookworm_conf_score',
        'schistosoma_conf_score']].values,
    [0,1,2,3],df_calibration[['pred_obj_id']].values
)
cc_std = ConformalClassifier()
cc_std.fit(alphas_cal)
display(cc_std)

ConformalClassifier(fitted=True, mondrian=False)

## Testing stage

### Applying the hinge on the testing dataset

In [11]:
conf_score_columns = [
    'ascaris_conf_score',
    'trichuris_conf_score',
    'hookworm_conf_score',
    'schistosoma_conf_score']
hinge_score_cols = [
    'ascaris_hinge_score', 
    'trichuris_hinge_score', 
    'hookworm_hinge_score', 
    'schistosoma_hinge_score']
df_testing = calculate_hinge_scores(
    df_testing, 
    conf_score_columns, 
    hinge_score_cols)

C:\Users\M\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
p_value_cols = [
    'ascaris_p_value', 
    'trichuris_p_value', 
    'hookworm_p_value', 
    'schistosoma_p_value']
df_testing = calculate_p_values(
    df_testing, 
    hinge_score_cols, 
    p_value_cols, 
    cc_std)

C:\Users\M\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [14]:
p_set_cols = [
    'ascaris_p_set', 
    'trichuris_p_set', 
    'hookworm_p_set', 
    'schistosoma_p_set']
df_testing = calculate_prediction_sets(
    df_testing, 
    hinge_score_cols, 
    p_set_cols, cc_std)

In [15]:
df_testing

,filename,image_width,image_height,gt_obj_id,gt_x_center,gt_y_center,gt_obj_width,gt_obj_height,pred_obj_id,pred_x_center,...,hookworm_hinge_score,schistosoma_hinge_score,ascaris_p_value,trichuris_p_value,hookworm_p_value,schistosoma_p_value,ascaris_p_set,trichuris_p_set,hookworm_p_set,schistosoma_p_set
0,11184288_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,3,0.9605,0.6972,0.0773,0.1583,3,0.961328,...,0.997449,0.395887,0.000320,0.000645,0.002307,0.181154,0,0,0,1
1,11184288_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,0,0.4059,0.4347,0.0586,0.1278,2,0.406641,...,0.613458,0.990231,0.005340,0.000493,0.047515,0.000307,0,0,1,0
2,11184288_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,-,-,-,-,-,1,0.405078,...,0.753833,0.984610,0.000203,0.132260,0.003669,0.000786,0,1,0,0
3,11184394_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,3,0.1703,0.7111,0.0984,0.125,3,0.16875,...,0.996007,0.278384,0.003881,0.002086,0.005779,0.364604,0,0,0,1
4,11184394_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,-,-,-,-,-,0,0.692187,...,0.997687,0.999338,0.403477,0.004441,0.002012,0.001533,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,9354474_1379_31_75602595-167c-4758-8f96-e8ee5e...,1280,720,0,0.6988,0.5833,0.0602,0.0833,0,0.698828,...,0.996884,0.999090,0.863466,0.003998,0.000810,0.001252,1,0,0,0
11870,9354474_1379_31_75602595-167c-4758-8f96-e8ee5e...,1280,720,0,0.4684,0.2347,0.0586,0.1028,0,0.46875,...,0.993931,0.998322,0.964677,0.004820,0.001173,0.001090,1,0,0,0
11871,9357486_1384_31_941af06b-54a3-4552-b75f-4e6e72...,1280,720,0,0.0187,0.7007,0.0375,0.0958,0,0.019141,...,0.993028,0.993571,0.659992,0.001110,0.000665,0.000495,1,0,0,0
11873,9419118_1393_31_55d950b3-e8d5-48b7-b7a7-30014f...,1280,720,-,-,-,-,-,3,0.970312,...,0.911459,0.584649,0.002398,0.001574,0.002199,0.060587,0,0,0,1


In [16]:
df_testing.to_csv('hinge_conf_score.csv',index=False)